In [1]:
import pandas as pd
import geopandas as gpd
import geojsonio
import plotly
import config

import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
plotly.tools.set_credentials_file(
    username=config.plotly_username, 
    api_key=config.plotly_api_key)
mapbox_access_token = config.mapbox_access_token

In [3]:
seats = gpd.read_file("data/parliamentary-boundaries-2018-working.geojson")

In [4]:
seats.head()

,KodPAR,Parliament,State,geometry
0,054,GERIK,PERAK,(POLYGON ((100.9933944673964 5.507324842759963...
1,055,LENGGONG,PERAK,(POLYGON ((100.9293339657613 5.514131158215433...
2,056,LARUT,PERAK,(POLYGON ((100.8672078753412 5.428777928590932...
3,058,BAGAN SERAI,PERAK,(POLYGON ((100.6241018705403 5.141014699246791...
4,057,PARIT BUNTAR,PERAK,(POLYGON ((100.4127561987131 4.992960828699954...


In [5]:
seats_json = seats[:20].to_json()

In [8]:
_ = geojsonio.display(seats_json)

In [9]:
data = go.Data([
    go.Scattermapbox(
        lat=['4.0891679'],
        lon=['100.5595715'],
        mode='markers',
    )
])
layout = go.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = 'data/single.geojson',
                type = 'fill',
                color = 'rgba(40,0,113,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=4.0891679,
            lon=100.5595715
        ),
        pitch=0,
        zoom=5.2,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='county-level-choropleths-python')

In [10]:
data = go.Data([
    go.Scattermapbox(
        lat=['45.5017'],
        lon=['-73.5673'],
        mode='markers',
    )
])
layout = go.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = 'data/florida-red-data-original.json',
                type = 'fill',
                color = 'rgba(163,22,19,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=27.8,
            lon=-83
        ),
        pitch=0,
        zoom=5.2,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='county-level-choropleths-python')